Before we get started, a couple of reminders to keep in mind when using iPython notebooks:

- Remember that you can see from the left side of a code cell when it was last run if there is a number within the brackets.
- When you start a new notebook session, make sure you run all of the cells up to the point where you last left off. Even if the output is still visible from when you ran the cells in your previous session, the kernel starts in a fresh state so you'll need to reload the data, etc. on a new session.
- The previous point is useful to keep in mind if your answers do not match what is expected in the lesson's quizzes. Try reloading the data and run all of the processing steps one by one in order to make sure that you are working with the same variables and data that are at each quiz stage.


## Load Data from CSVs

In [1]:
import unicodecsv

## Longer version of code (replaced with shorter, equivalent version below)

# enrollments = []
# f = open('enrollments.csv', 'rb')
# reader = unicodecsv.DictReader(f)
# for row in reader:
#     enrollments.append(row)
# f.close()

with open('enrollments.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)
    enrollments = list(reader)

In [4]:
#####################################
#                 1                 #
#####################################

## Read in the data from daily_engagement.csv and project_submissions.csv 
## and store the results in the below variables.
## Then look at the first row of each table.

with open('daily_engagement.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)
    daily_engagement = list(reader)
print(daily_engagement[0])

with open('project_submissions.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)
    project_submissions = list(reader)
print(project_submissions[0])

OrderedDict([('acct', '0'), ('utc_date', '2015-01-09'), ('num_courses_visited', '1.0'), ('total_minutes_visited', '11.6793745'), ('lessons_completed', '0.0'), ('projects_completed', '0.0')])
OrderedDict([('creation_date', '2015-01-14'), ('completion_date', '2015-01-16'), ('assigned_rating', 'UNGRADED'), ('account_key', '256'), ('lesson_key', '3176718735'), ('processing_state', 'EVALUATED')])


## Fixing Data Types

In [5]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

OrderedDict([('account_key', '448'),
             ('status', 'canceled'),
             ('join_date', datetime.datetime(2014, 11, 10, 0, 0)),
             ('cancel_date', datetime.datetime(2015, 1, 14, 0, 0)),
             ('days_to_cancel', 65),
             ('is_udacity', True),
             ('is_canceled', True)])

In [6]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

OrderedDict([('acct', '0'),
             ('utc_date', datetime.datetime(2015, 1, 9, 0, 0)),
             ('num_courses_visited', 1),
             ('total_minutes_visited', 11.6793745),
             ('lessons_completed', 0),
             ('projects_completed', 0)])

In [7]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

OrderedDict([('creation_date', datetime.datetime(2015, 1, 14, 0, 0)),
             ('completion_date', datetime.datetime(2015, 1, 16, 0, 0)),
             ('assigned_rating', 'UNGRADED'),
             ('account_key', '256'),
             ('lesson_key', '3176718735'),
             ('processing_state', 'EVALUATED')])

Note when running the above cells that we are actively changing the contents of our data variables. If you try to run these cells multiple times in the same session, an error will occur.

## Investigating the Data

In [25]:
#####################################
#                 2                 #
#####################################

## Find the total number of rows and the number of unique students (account keys)
## in each table.

enrollment_num_rows = len(enrollments)
enrollment_keys = []
for row in enrollments:
    if row['account_key'] not in enrollment_keys:
        enrollment_keys.append(row['account_key'])
enrollment_num_unique_students = len(enrollment_keys)
print('enrollments:', enrollment_num_unique_students, '/', enrollment_num_rows)

engagement_num_rows = len(daily_engagement)
engagement_keys = []
for row in daily_engagement:
    if row['acct'] not in engagement_keys:
        engagement_keys.append(row['acct'])
engagement_num_unique_students = len(engagement_keys)
print('daily_engagement:', engagement_num_unique_students, '/', engagement_num_rows)

submission_num_rows = len(project_submissions)
submission_keys = []
for row in project_submissions:
    if row['account_key'] not in submission_keys:
        submission_keys.append(row['account_key'])
submission_num_unique_students = len(submission_keys)
print('project_submissions:', submission_num_unique_students, '/', submission_num_rows)


enrollments: 1302 / 1640
daily_engagement: 1237 / 136240
project_submissions: 743 / 3642


## Problems in the Data

In [26]:
#####################################
#                 3                 #
#####################################

## Rename the "acct" column in the daily_engagement table to "account_key".

for row in daily_engagement:
    row['account_key'] = row['acct']
    del row['acct']
print(daily_engagement[0]['account_key'])

0


## Missing Engagement Records

In [30]:
#####################################
#                 4                 #
#####################################

## Find any one student enrollments where the student is missing from the daily engagement table.
## Output that enrollment.

for enrollment in enrollments:
    if enrollment['account_key'] not in engagement_keys:
        print(enrollment)
        break

OrderedDict([('account_key', '1219'), ('status', 'canceled'), ('join_date', datetime.datetime(2014, 11, 12, 0, 0)), ('cancel_date', datetime.datetime(2014, 11, 12, 0, 0)), ('days_to_cancel', 0), ('is_udacity', False), ('is_canceled', True)])


## Checking for More Problem Records

In [47]:
#####################################
#                 5                 #
#####################################

## Find the number of surprising data points (enrollments missing from
## the engagement table) that remain, if any.
count=0
for enrollment in enrollments:
    if enrollment['account_key'] not in engagement_keys and (enrollment['days_to_cancel'] is None or enrollment['days_to_cancel'] > 0):
        count+=1
        print(enrollment)
print(count)

OrderedDict([('account_key', '1304'), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 1, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 3, 10, 0, 0)), ('days_to_cancel', 59), ('is_udacity', True), ('is_canceled', True)])
OrderedDict([('account_key', '1304'), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 3, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 6, 17, 0, 0)), ('days_to_cancel', 99), ('is_udacity', True), ('is_canceled', True)])
OrderedDict([('account_key', '1101'), ('status', 'current'), ('join_date', datetime.datetime(2015, 2, 25, 0, 0)), ('cancel_date', None), ('days_to_cancel', None), ('is_udacity', True), ('is_canceled', False)])
3


## Tracking Down the Remaining Problems

In [48]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

In [51]:
# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [52]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print(len(non_udacity_enrollments))
print(len(non_udacity_engagement))
print(len(non_udacity_submissions))

1622
135656
3634


## Refining the Question

In [70]:
#####################################
#                 6                 #
#####################################

## Create a dictionary named paid_students containing all students who either
## haven't canceled yet or who remained enrolled for more than 7 days. The keys
## should be account keys, and the values should be the date the student enrolled.

paid_students = {}
for enrollment in non_udacity_enrollments:
    if enrollment['cancel_date'] is None or (enrollment['cancel_date'] - enrollment['join_date']).days > 7:
        key = enrollment['account_key']
        val = enrollment['join_date']
        if key not in paid_students or val > paid_students[key]:
            paid_students[key] = val
print(len(paid_students))

995


## Getting Data from First Week

In [98]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7 and time_delta.days >= 0

In [99]:
#####################################
#                 7                 #
#####################################

## Create a list of rows from the engagement table including only rows where
## the student is one of the paid students you just found, and the date is within
## one week of the student's join date.

paid_engagement_in_first_week = []
for engagement in non_udacity_engagement:
    try:
        join = paid_students[engagement['account_key']]
        if within_one_week(join, engagement['utc_date']):
            paid_engagement_in_first_week.append(engagement)
    except:
        continue
print(len(paid_engagement_in_first_week))

6919


## Exploring Student Engagement

In [100]:
from collections import defaultdict

# Create a dictionary of engagement grouped by student.
# The keys are account keys, and the values are lists of engagement records.
engagement_by_account = defaultdict(list)
for engagement_record in paid_engagement_in_first_week:
    account_key = engagement_record['account_key']
    engagement_by_account[account_key].append(engagement_record)

In [101]:
# Create a dictionary with the total minutes each student spent in the classroom during the first week.
# The keys are account keys, and the values are numbers (total minutes)
total_minutes_by_account = {}
for account_key, engagement_for_student in engagement_by_account.items():
    total_minutes = 0
    for engagement_record in engagement_for_student:
        total_minutes += engagement_record['total_minutes_visited']
    total_minutes_by_account[account_key] = total_minutes

In [102]:
import numpy as np

# Summarize the data about minutes spent in the classroom
total_minutes = []
for key, val in total_minutes_by_account.items():
    total_minutes.append(val)
print('Mean:{}'.format(np.mean(np.asarray(total_minutes))))
print('Standard deviation:{}'.format(np.std(total_minutes)))
print('Minimum:{}'.format(np.min(total_minutes)))
print('Maximum:{}'.format(np.max(total_minutes)))

Mean:306.70832675342825
Standard deviation:412.99693340852957
Minimum:0.0
Maximum:3564.7332644989997


## Debugging Data Analysis Code

In [103]:
#####################################
#                 8                 #
#####################################

## Go through a similar process as before to see if there is a problem.
## Locate at least one surprising piece of data, output it, and take a look at it.
week = 7*24*60
print(week)
print()
for account_key, val in total_minutes_by_account.items():
    if val > week:
        for item in engagement_by_account[account_key]:
            print(item)
        print(val)

10080



## Lessons Completed in First Week

In [110]:
#####################################
#                 9                 #
#####################################

## Adapt the code above to find the mean, standard deviation, minimum, and maximum for
## the number of lessons completed by each student during the first week. Try creating
## one or more functions to re-use the code above.

lessons_completed_by_account = {}
for account_key, engagement_for_student in engagement_by_account.items():
    lessons_completed = 0
    for engagement_record in engagement_for_student:
        lessons_completed += engagement_record['lessons_completed']
    lessons_completed_by_account[account_key] = lessons_completed

lessons_completed = []
for key, val in lessons_completed_by_account.items():
    lessons_completed.append(val)
print('Mean:{}'.format(np.mean(np.asarray(lessons_completed))))
print('Standard deviation:{}'.format(np.std(lessons_completed)))
print('Minimum:{}'.format(np.min(lessons_completed)))
print('Maximum:{}'.format(np.max(lessons_completed)))


Mean:1.636180904522613
Standard deviation:3.002561299829423
Minimum:0
Maximum:36


## Number of Visits in First Week

In [142]:
######################################
#                 10                 #
######################################

## Find the mean, standard deviation, minimum, and maximum for the number of
## days each student visits the classroom during the first week.

days_visited_by_account = {}
for account_key, engagement_for_student in engagement_by_account.items():
    days_visited = []
    for engagement_record in engagement_for_student:
        print(engagement_record)
        if engagement_record['num_courses_visited'] is 0:
            continue
        day = engagement_record['utc_date'].strftime('%d')
        if day not in days_visited:
            days_visited.append(day)
    days_visited_by_account[account_key] = len(days_visited)
    break

days_visited = []
for key, val in days_visited_by_account.items():
    days_visited.append(val)
print('Mean:{}'.format(np.mean(np.asarray(days_visited))))
print('Standard deviation:{}'.format(np.std(days_visited)))
print('Minimum:{}'.format(np.min(days_visited)))
print('Maximum:{}'.format(np.max(days_visited)))


OrderedDict([('utc_date', datetime.datetime(2015, 1, 9, 0, 0)), ('num_courses_visited', 1), ('total_minutes_visited', 11.6793745), ('lessons_completed', 0), ('projects_completed', 0), ('account_key', '0')])
OrderedDict([('utc_date', datetime.datetime(2015, 1, 10, 0, 0)), ('num_courses_visited', 2), ('total_minutes_visited', 37.2848873333), ('lessons_completed', 0), ('projects_completed', 0), ('account_key', '0')])
OrderedDict([('utc_date', datetime.datetime(2015, 1, 11, 0, 0)), ('num_courses_visited', 2), ('total_minutes_visited', 53.6337463333), ('lessons_completed', 0), ('projects_completed', 0), ('account_key', '0')])
OrderedDict([('utc_date', datetime.datetime(2015, 1, 12, 0, 0)), ('num_courses_visited', 1), ('total_minutes_visited', 33.4892696667), ('lessons_completed', 0), ('projects_completed', 0), ('account_key', '0')])
OrderedDict([('utc_date', datetime.datetime(2015, 1, 13, 0, 0)), ('num_courses_visited', 1), ('total_minutes_visited', 64.7796776667), ('lessons_completed', 0),

## Splitting out Passing Students

In [143]:
######################################
#                 11                 #
######################################

## Create two lists of engagement data for paid students in the first week.
## The first list should contain data for students who eventually pass the
## subway project, and the second list should contain data for students
## who do not.

subway_project_lesson_keys = ['746169184', '3176718735']

for p in paid_engagement_in_first_week:
    print(p)
    break

passing_engagement = []
non_passing_engagement = []
for account_key, engagement_for_student in engagement_by_account.items():
    break
    days_visited = []
    for engagement_record in engagement_for_student:
        if engagement_record['num_courses_visited'] is 0:
            continue
        day = engagement_record['utc_date'].strftime('%d')
        if day not in days_visited:
            days_visited.append(day)
    days_visited_by_account[account_key] = len(days_visited)

print(len(passing_engagement))
print(len(non_passing_engagement))

OrderedDict([('utc_date', datetime.datetime(2015, 1, 9, 0, 0)), ('num_courses_visited', 1), ('total_minutes_visited', 11.6793745), ('lessons_completed', 0), ('projects_completed', 0), ('account_key', '0')])
0
0


## Comparing the Two Student Groups

In [165]:
######################################
#                 12                 #
######################################

## Compute some metrics you're interested in and see how they differ for
## students who pass the subway project vs. students who don't. A good
## starting point would be the metrics we looked at earlier (minutes spent
## in the classroom, lessons completed, and days visited).

'''
    Note: daily_engagement_full.csv required
'''


'\n    Note: daily_engagement_full.csv required\n'

## Making Histograms

In [163]:
######################################
#                 13                 #
######################################

## Make histograms of the three metrics we looked at earlier for both
## students who passed the subway project and students who didn't. You
## might also want to make histograms of any other metrics you examined.

%matplotlib inline
import matplotlib.pyplot as plt
#plt.hist(...)

## Improving Plots and Sharing Findings

In [ ]:
######################################
#                 14                 #
######################################

## Make a more polished version of at least one of your visualizations
## from earlier. Try importing the seaborn library to make the visualization
## look better, adding axis labels and a title, and changing one or more
## arguments to the hist() function.